In [1]:
import os
import pandas as pd

In [50]:
# function for reading the data files
def build_data_frame(path):
    rows = []
    index = []
    classification =[]
    for file_name, text, classification in read_files(path):
        rows.append({'text':text, 'class': classification})
        index.append(file_name)
        
    data_frame = pd.DataFrame(rows,index=index)
    return data_frame

In [51]:
def read_files(path):
    newline=''
    for root, dir_names, file_names in os.walk(path):
        print('Root folder: {0}'.format(root))
        print('Number of files read: {0}'.format(len(file_names)))
        for file_name in file_names:
            file_path = os.path.join(root,file_name)
            if(os.path.isfile(file_path)):
                #print(file_name)
                if("D" in file_name):
                    label="D"
                elif("R" in file_name):
                    label="R"
                else:
                    label="X"
                lines = []
                f = open(file_path)
                for line in f:
                    lines.append(line.rstrip("\n"))
                f.close()
                content=newline.join(lines)
                yield file_name, content, label

In [58]:
# here I set the path of data set using os.getcwd()
path = os.path.join(os.getcwd(), 'data_set')
# intitalize the empty data frame
data = pd.DataFrame({'text':[],'class':[]})
# call the function to build the data set
data=data.append(build_data_frame(path))

Root folder: /home/sadu/Documents/IE594-Data-Science/Speech-classification/data_set
Number of files read: 856


In [53]:
df[['number','time']] = df['duration'].str.extract(r'(\d+)\s*(\w+)', expand=True)

466

In [122]:
data['file_name']=data.index.values
[x for y in data['file_name'] for x,y,z,k in re.split('_',y)]

ValueError: not enough values to unpack (expected 4, got 3)

In [98]:
a = '1,2,3'
len(a)

5

In [75]:
import re
apply(re.split('_',)[0], axis = 1)
test = 'Static_1264__1264_11232013_1234'
re.split('_',test)[1]

'1264'

In [56]:
data.columns

Index(['class', 'text'], dtype='object')

In [57]:
data[data["class"]=="R"].shape

(466, 2)